In [2]:
import requests
from lxml import html 
import pandas as pd 
import smtplib #smtp服务器
from email.mime.text import MIMEText #邮件文本
from email.header import Header
import time


def Get_Condtion():
    House_Type = input('请输入房屋类型编号，并用英文逗号隔开，编号对应如下：独立屋--33，联排屋--34，公寓--35，后巷屋--36')
    Rent_Form = input('请输入出租形式的编号，并用英文逗号隔开，编号对应如下：分组--40，整租--42')
    Price_Level = input('请输入价格区间，并用英文逗号隔开')
    Num_Of_Bedroom = input('请选择卧室的数量对应的编号，并用逗号隔开，编号对应如下：1卧+--56，2卧+--57，3卧+--58，4卧+--59，5卧+--60')
    Rent_Include = input('请选择租金包含的项目对应的编号，并用逗号隔开，编号对应如下：水费--44，网络--45，天然气--46，电费--316')
    Equipment = input('请选择需要的设备对应的编号，并用逗号隔开，编号对应如下：洗衣设施--49，独立厨房--50，基本家具--51，基本电器--52，独立车位--53，独立卫浴--54，中央空调--394，充电桩--395')
    Others = input('请选择需要的其他条件，并用逗号隔开，对应编号如下：可养猫--392，可养狗--393')
    return [House_Type,Rent_Form,Price_Level,Num_Of_Bedroom,Rent_Include,Equipment,Others]

def Condition_Url(Condition_List):
    Id = ['32=','39=','Price=','55=','43=','48=','391=']
    for i in range(len(Id)):
        Id[i] = Id[i] + Condition_List[i]
    Url = '&'.join(i for i in Id)
    Final_Url = 'https://c.vanpeople.com/zufang?' + Url
    return Final_Url

def Crawl_By_Condition(Url):
    Rent_Info = pd.DataFrame()
    Page_num = 1
    headers = {'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
           'Accept-Encoding':'gzip, deflate, br',
           'Accept-Language':'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
           'Connection':'keep-alive',
           'Cookie':'PHPSESSID=62150fpvjloapn1435mqagbkjp; _ga=GA1.2.1798208994.1664753544; IcYL_be21_saltkey=hM7YZtY7; IcYL_be21_lastvisit=1672700769; IcYL_be21_sid=Wa06lC; IcYL_be21_lastact=1672704369%09get_uid_ajax.php%09; _gid=GA1.2.91719644.1672704370; _hjSessionUser_1585734=eyJpZCI6ImJhY2ExYzVjLTVkOGQtNTA2Yi04MTEzLTE5NTVkOTQyZGNjNyIsImNyZWF0ZWQiOjE2NzI3MDQzNzA0NDUsImV4aXN0aW5nIjpmYWxzZX0=; PHPSESSID=62150fpvjloapn1435mqagbkjp; country_code_name=CA',
           'Host':'c.vanpeople.com',
           'sec-ch-ua':'"Not?A_Brand";v="8", "Chromium";v="108", "Microsoft Edge";v="108"',
           'sec-ch-ua-mobile':'?0',
           'sec-ch-ua-platform':'"Windows"',
           'Sec-Fetch-Dest': 'document',
           'Sec-Fetch-Mode':'navigate',
           'Sec-Fetch-Site':'none',
           'Sec-Fetch-User':'?1',
           'Upgrade-Insecure-Requests':'1',
           'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.54'}
    while(1):
        Current_Url = Url + '&' + 'page=' + str(Page_num)
        source = requests.get(Current_Url,headers=headers).content.decode()
        page = html.fromstring(source)
        item_url = page.xpath('//li[@class="list "]/@data-url')
        title = page.xpath('//li[@class="list "]/div/div/a[@class="c-list-title"]/text()')
        money = page.xpath('//li[@class="list "]/div/div/div/span[@class="money"]/text()')
        update = page.xpath('//li[@class="list "]/div/div/span[@class="c-list-date"]/text()')
        data = {
            'title':title,
            'money':money,
            'item_url':item_url,
            'update':update
        }
        data = pd.DataFrame(data)
        Rent_Info = Rent_Info.append(data)

        if data['update'].str.contains('天前').any():
            break
        else:
            Page_num += 1

    return Rent_Info

def send_email(subject="24zbw推送提醒",content="24zbw推送集锦录像失败，请查看",recver="test@outlook.com"):
    # 第三方 SMTP 服务
    mail_host="smtp.office365.com"  #设置服务器
    mail_user="yize_liu000407@outlook.com"    #用户名
    mail_pass="Prime2357"   #口令 
    sender = 'yize_liu000407@outlook.com'
    port = 587
    # 调试开启 写死 我的邮件
    receivers = recver # 接收邮件，可设置为你的QQ邮箱或者其他邮箱
    message = MIMEText(content, 'plain', 'utf-8')
    message['From'] = sender
    message['To'] =  ';'.join(recver)
    subject = subject
    message['Subject'] = Header(subject, 'utf-8')
    with smtplib.SMTP("smtp.office365.com", port) as server:
      server.starttls() # Secure the connection
      server.login(mail_user, mail_pass)
      server.sendmail(sender, recver, message.as_string())
      print("mail successfully sent")

def main():
    condition = Get_Condtion()
    Url = Condition_Url(condition)
    Rent_Info = Crawl_By_Condition(Url)

    if len(Rent_Info) == 0:
        print('暂时没有符合条件的房子，如果后续有更新会再发送给你')
    else:
        display(Rent_Info)

    while(1):
        New_Info = Crawl_By_Condition(Url)
        if len(New_Info) > len(Rent_Info):
            Info_diff = pd.concat([New_Info,Rent_Info]).drop_duplicates(keep=False)
            print('您想找的房源有最近发布信息')
            display(Info_diff)
        Rent_Info = New_Info
        time.sleep(60)


main()

C:\Users\yize_\AppData\Local\Temp\ipykernel_42372\142765383.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Rent_Info = Rent_Info.append(data)


,title,money,item_url,update
0,全新Richmond一室一厅出租,"$2,550",/zufang/item-2952979.html,2022/10/24
1,高贵林中心区域两房一Den两卫,"$3,200",/zufang/item-2940096.html,2022/09/19
2,Brentwood 全新两房出租,"$3,200",/zufang/item-2987629.html,1小时前
3,Metrotown中心高档公寓中央空调,"$3,200",/zufang/item-2993358.html,昨天
4,温哥华高级公寓三房两卫出租,"$3,995",/zufang/item-2984416.html,昨天
5,【全新】列治文商场 2房 2卫,"$3,100",/zufang/item-2977444.html,昨天
6,近SFU/CC天车站 3室2卫包家具$3000,"$3,000",/zufang/item-2992988.html,昨天
7,UBC校内 一室公寓 全包 拎包入住,"$3,000",/zufang/item-2990272.html,昨天
8,南素里公寓楼2室2卫1书房出租,"$2,650",/zufang/item-2990189.html,昨天
9,列治文中心2 房月租,"$2,800",/zufang/item-2992962.html,昨天


C:\Users\yize_\AppData\Local\Temp\ipykernel_42372\142765383.py:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Rent_Info = Rent_Info.append(data)
